# GPAO II

We have pre-implemented automation code that does OIM addition to target energy adjusting. We use `GPAO` wrapper class that uses `DAO` as `real_finder`. 

Unfortunately, this process generates folder in the user's directory rather than put out info on the `sys.io`. That is because it genrates a lot of info and 

In [3]:
import numpy as np
from taps.paths import Paths
from taps.models import MullerBrown, Gaussian
from taps.pathfinder import DAO, GPAO
from taps.coords import Cartesian
from taps.visualize import view
from taps.ml.regressions import Regression
from taps.projectors import Sine
from taps.db import ImageDatabase

# We are going to use only the 30% of the coordinate information
name = "AuGPAO"
#trialn = "test9/"
trialn = "test/"
#directory = name + '/'
directory = './'


N = 300

x = np.linspace(-0.55822365, 0.6234994, N)
y = np.linspace(1.44172582, 0.02803776, N)

coords = Cartesian(coords=np.array([x, y]))

Nk = N - 270
prj = Sine(N=N, Nk=Nk, 
           init=coords.coords[..., 0].copy(), 
           fin=coords.coords[..., -1].copy())



action_kwargs = {
    'Onsager Machlup':{    
        'gam': 1.,
    },
    'Energy Restraint':{
        'muE': 1.,
        'Et': -0.45
    }
}

search_kwargs = {"method":"L-BFGS-B"}




finder = DAO(action_kwargs=action_kwargs,
             search_kwargs=search_kwargs, 
             prj=prj)

maxtrial=200

plot_kwargs = {"savefig":True, "gaussian": True, 'viewer': 'MullerBrown'}

finder = GPAO(real_finder=finder, maxtrial=maxtrial, 
              logfile=directory+trialn + name +'.log',
              label= directory + trialn + name,
              phase_kwargs={"Auto Et":{'Et_tol':0.05, 'cov_tol':0.01, 'Etol': 0.05}, 
                                "Auto Et":{'Et_tol':0.05, 'cov_tol':0.01, 'Etol': 0.05}},
                  plot_kwargs=plot_kwargs)


imgdata = ImageDatabase(filename="mullerbrown.db")


model = MullerBrown()

bounds = np.array([[1e-2, 5e1],   # sigma_f': 0
                    [1e-2, 1.],   # l^2': 1
                    [1e-6, 1e-5],  # sigma_n^e': 2
                    [1e-7, 1e-6]])# sigma_n^f': 3

regression = Regression(kernel_regression_kwargs={'method': 'L-BFGS-B',
                                                 'bounds': bounds})
model = Gaussian(real_model=model, regression=regression)

paths = Paths(coords=coords, model=model, finder=finder, imgdata=imgdata)

paths.add_data(index=[0, coords.N//3, -1])
paths.search()


AttributeError: 'float' object has no attribute 'reshape'

In [3]:
pdb.write([{'paths': paths}])

[1]

In [4]:
paths.get_masses()

1.0

In [5]:
aa = pdb.read(ids=[1])